In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from src import RGBNirModule, RGBNirDataModule

dm = RGBNirDataModule(batch_size=512, num_workers=10, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [13]:
model = RGNirModule.load_from_checkpoint('checkpoints/r18da-val_loss=6.45633-epoch=4.ckpt')
model.hparams

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/juan/Desktop/competis/GeoLifeCLEF2022/src/ds.py", line 68, in __getitem__
    img = self.trans(image=img)['image']
  File "/home/juan/miniconda3/lib/python3.8/site-packages/albumentations/core/composition.py", line 194, in __call__
    if self.is_check_args:
AttributeError: 'Compose' object has no attribute 'is_check_args'


In [ ]:
from tqdm import tqdm
from src.GLC.metrics import top_30_error_rate
import numpy as np 

model.cuda(1)
dl = dm.val_dataloader()
accs = []
for imgs, labels in tqdm(dl):
    preds = model.predict(imgs)
    accs.append(top_30_error_rate(labels, preds.cpu()))
np.mean(accs)

100%|██████████| 79/79 [01:51<00:00,  1.41s/it]


0.7366962245077356

In [ ]:
from tqdm import tqdm

model.cuda(1)
dl = dm.test_dataloader()
labels, observations = [], []
for imgs, observation_ids in tqdm(dl):
    preds = model.predict(imgs)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observations += observation_ids.numpy().tolist()

100%|██████████| 72/72 [00:21<00:00,  3.38it/s]


In [ ]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
23252,20647561,4918 1352 10393 5462 6885 5406 5855 7494 8913 ...
19542,20250188,4940 4991 5483 1450 5 6061 5143 337 5940 5458 ...
20448,20347366,1227 2187 2733 5175 2071 2561 5965 5618 1597 9...
12025,10299709,727 1025 373 235 576 85 514 399 47 316 591 835...
16694,10731885,21 216 207 125 694 213 395 338 278 683 496 333...


In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
869,10041161,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
34320,21840242,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
29448,21320097,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
24813,20822365,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4325,10204759,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3778,10178363,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
19217,20213545,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
7739,10367312,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
22308,20544460,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
6521,10308473,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
